In [1]:
%cd ..

/Users/danieloliveiradebrito/Projetos/causal-inference


## 1.1. Foundation of Linear Regression

Example 1.1.1: approx a Smooth Function with a Polynomail Dictionary

In [2]:
import numpy as np
import altair as alt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
def create_X(p: int, w: np.array) -> np.ndarray:

    X = w

    while p - 1 > 1:
        X = np.c_[X, (w ** (p - 1))] 
        p -= 1

    return sm.add_constant(X)

def fit_ols(X: np.array, g: np.array, w: np.array) -> pd.DataFrame:
    model = sm.OLS(g, X)
    res = model.fit()
    predicted = res.get_prediction().summary_frame()["mean"].values

    return pd.DataFrame(
        dict(
            w = w,
            y_true = g,
            y_pred = predicted
        )
    )

def plot_predictions(df: pd.DataFrame, p: int) -> alt.Chart:
    color_scale =  alt.Scale(
                    domain = ["y_true", "y_pred"],
                    range = ["blue", "orange"])


    return alt.Chart(
        df.melt(id_vars = "w"),
        title = f"p = {p}"
    ).mark_line().encode(x = alt.X("w", title = "w"),
                        y = alt.Y("value", title = "Values"),
                        color = alt.Color("variable", scale = color_scale, legend=None),
                        strokeDash = alt.condition(
                            alt.datum.variable == "y_pred",
                            alt.value([5, 5]),
                            alt.value([0, 0]),
                            legend=None),
                        ).configure_axis(grid = False)

In [4]:
w = np.arange(0, 1, .01)

g = np.exp(4 * w)

for p in [2, 3, 4]:
    X = create_X(p, w)

    chart = (plot_predictions(fit_ols(X, g, w), p))

    chart.display()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## 1.2. Statistical Properties

In [5]:
n = 100
p = int( n / 2)

y = np.random.normal(0, 1, size=[n, 1])
X = np.random.normal(0, scale=1, size=[n, p])

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.551
Model:                            OLS   Adj. R-squared (uncentered):              0.102
Method:                 Least Squares   F-statistic:                              1.227
Date:                Wed, 06 Mar 2024   Prob (F-statistic):                       0.236
Time:                        14:31:34   Log-Likelihood:                         -95.695
No. Observations:                 100   AIC:                                      291.4
Df Residuals:                      50   BIC:                                      421.6
Df Model:                          50                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1594      0.136      1.174      0.246      -0.113       0.432
x2             0.0451      0.152      0.297      0.768      -0.260       0.350
x3             0.0403      0.134      0.300      0.765      -0.230       0.310
x4             0.0834      0.122      0.682      0.498      -0.162       0.329
x5             0.0560      0.114      0.491      0.626      -0.173       0.285
x6            -0.1228      0.113     -1.088      0.282      -0.349       0.104
x7             0.3955      0.166      2.386      0.021       0.063       0.728
x8             0.2093      0.146      1.432      0.158      -0.084       0.503
x9             0.0855      0.133      0.642      0.524      -0.182       0.353
x10           -0.0598      0.135     -0.444      0.659      -0.330       0.211
x11            0.1571      0.117      1.344      0.185      -0.078       0.392
x12            0.0904      0.105      0.861      0.393      -0.121       0.301
x13           -0.1034      0.148     -0.699      0.488      -0.400       0.194
x14           -0.0065      0.154     -0.043      0.966      -0.315       0.302
x15            0.0243      0.142      0.171      0.865      -0.261       0.309
x16           -0.1134      0.135     -0.838      0.406      -0.385       0.159
x17           -0.2035      0.132     -1.547      0.128      -0.468       0.061
x18           -0.2690      0.177     -1.520      0.135      -0.624       0.086
x19            0.4047      0.123      3.297      0.002       0.158       0.651
x20           -0.2276      0.151     -1.505      0.139      -0.531       0.076
x21           -0.0395      0.162     -0.245      0.808      -0.364       0.285
x22            0.1328      0.125      1.064      0.293      -0.118       0.383
x23            0.1486      0.160      0.927      0.358      -0.173       0.471
x24           -0.2975      0.164     -1.812      0.076      -0.627       0.032
x25            0.1012      0.135      0.749      0.457      -0.170       0.373
x26           -0.0263      0.110     -0.239      0.812      -0.247       0.194
x27           -0.0014      0.100     -0.014      0.989      -0.203       0.200
x28            0.1048      0.150      0.700      0.487      -0.196       0.405
x29           -0.0931      0.111     -0.839      0.405      -0.316       0.130
x30           -0.0047      0.125     -0.038      0.970      -0.256       0.247
x31            0.1109      0.137      0.812      0.420      -0.163       0.385
x32           -0.0379      0.118     -0.322      0.748      -0.274       0.198
x33           -0.0727      0.143     -0.507      0.614      -0.361       0.215
x34            0.1433      0.132      1.086      0.283      -0.122       0.408
x35           -0.1711      0.117     -1.465      0.149      -

## 1.4. Application

### Prediction of Wages

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
df = pd.read_csv(file)

In [8]:
df.describe()

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
count,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000
mean,23.410410,2.970787,0.444466,0.023301,0.243883,0.278058,0.317670,0.137087,0.259612,0.296505,0.216117,0.227767,13.760583,3.018925,8.235867,25.118038,5310.737476,11.670874,6629.154951,13.316893
std,21.003016,0.570385,0.496955,0.150872,0.429465,0.448086,0.465616,0.343973,0.438464,0.456761,0.411635,0.419432,10.609465,4.000904,14.488962,53.530225,11874.356080,6.966684,5333.443992,5.701019
min,3.021978,1.105912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,370.000000,2.000000
25%,13.461538,2.599837,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.250000,0.125000,0.062500,1740.000000,5.000000,4880.000000,9.000000
50%,19.230769,2.956512,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,1.000000,1.000000,4040.000000,13.000000,7370.000000,14.000000
75%,27.777778,3.324236,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,21.000000,4.410000,9.261000,19.448100,5610.000000,17.000000,8190.000000,18.000000
max,528.845673,6.270697,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,22.090000,103.823000,487.968100,100000.000000,22.000000,100000.000000,22.000000


In [9]:
df.shape

(5150, 20)

In [10]:
Y = np.log(df["wage"])
Z = df.drop(columns=["wage", "lwage"])
Z.shape

(5150, 18)

In [11]:
train, test = train_test_split(df, test_size = 0.20, random_state=123)

#### In sample

In [12]:
#" + ".join(Z.columns)

model_base = "lwage ~ sex + exp1 + shs + hsg + scl + clg + mw + so + we + C(occ2) + C(ind2)"
base = smf.ols(model_base, data=train)
results_base = base.fit()

rsquared_base = results_base.rsquared
rsquared_adj_base = results_base.rsquared_adj
mse_base = np.mean(results_base.resid ** 2)
mse_adj_base = results_base.mse_resid

print(f"R2 = {rsquared_base:.4f}")
print(f"R2 ajustado = {rsquared_adj_base:.4f}")
print(f"MSE = {mse_base:.4f}")
print(f"MSE ajustado = {mse_adj_base:.4f}")


R2 = 0.3176
R2 ajustado = 0.3092
MSE = 0.2202
MSE ajustado = 0.2229


In [13]:
#" + ".join(Z.columns)

flexible_model = "lwage ~ sex + shs + hsg + scl + C(occ2) + C(ind2) + mw + so + we  + (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg + C(occ2) + C(ind2) + mw + so + we)"
flex = smf.ols(flexible_model, data=train)
results_flex = flex.fit()

rsquared_flex = results_flex.rsquared
rsquared_adj_flex = results_flex.rsquared_adj
mse_flex = np.mean(results_flex.resid ** 2)
mse_adj_flex = results_flex.mse_resid

print(f"R2 = {rsquared_flex:.4f}")
print(f"R2 ajustado = {rsquared_adj_flex:.4f}")
print(f"MSE = {mse_flex:.4f}")
print(f"MSE ajustado = {mse_adj_flex:.4f}")


R2 = 0.3643
R2 ajustado = 0.3241
MSE = 0.2051
MSE ajustado = 0.2181


In [14]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [15]:
X = flex.data.exog[:, 1:]
y = flex.data.endog

lasso = Pipeline([
    ("scale", StandardScaler()),
    ("lasso", LassoCV())
])
lasso.fit(X, y)

/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2784518041252113, tolerance: 0.10708269218738481
  model = cd_fast.enet_coordinate_descent_gram(
/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1525919191144567, tolerance: 0.10708269218738481
  model = cd_fast.enet_coordinate_descent_gram(
/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3126010241782069, tolerance: 0.1070826921873848

Pipeline(steps=[('scale', StandardScaler()), ('lasso', LassoCV())])

In [16]:
n, p = X.shape
p += 1
mse_lasso = np.mean((y - lasso.predict(X)) ** 2)
mse_adj_lasso = mse_lasso * n / (n - p)
rsquared_lasso = 1 - mse_lasso / np.var(y)
rsquared_adj_lasso = 1 - mse_adj_lasso / np.var(y)

print(f"R2 = {rsquared_lasso:.4f}")
print(f"R2 ajustado = {rsquared_adj_lasso:.4f}")
print(f"MSE = {mse_lasso:.4f}")
print(f"MSE ajustado = {mse_adj_lasso:.4f}")

R2 = 0.3309
R2 ajustado = 0.2885
MSE = 0.2159
MSE ajustado = 0.2296


In [17]:
res_df = pd.DataFrame(
    dict(
        model = ["Basic reg", "Flexible reg", "Flexible Lasso"],
        p = [
            results_base.params.shape[0],
            results_flex.params.shape[0],
            results_flex.params.shape[0]
        ],
        r2 = [
            rsquared_base,
            rsquared_flex,
            rsquared_lasso
        ],
        mse = [mse_base, mse_flex, mse_lasso],
        adj_r2 = [rsquared_adj_base, rsquared_adj_flex, rsquared_adj_lasso],
        adj_mse = [mse_adj_base, mse_adj_flex, mse_adj_lasso]
    )
)

res_df

,model,p,r2,mse,adj_r2,adj_mse
0,Basic reg,51,0.317622,0.220187,0.309237,0.222947
1,Flexible reg,246,0.364346,0.205110,0.324146,0.218135
2,Flexible Lasso,246,0.330946,0.215888,0.288461,0.229597


#### Out of sample

In [18]:
tmp = smf.ols(model_base, data=df)
X = tmp.data.exog
y = tmp.data.endog
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)

In [19]:
reg_basic = sm.OLS(y_train, X_train).fit()
yhat_reg_base = reg_basic.predict(X_test)

MSE_test1 = sum((y_test - yhat_reg_base) ** 2) / y_test.shape[0]
R2_test1 = 1 - MSE_test1 / np.var(y_test)

print(f"Test MSE for basic model = {MSE_test1:.2f}")
print(f"Test R2 for basic model = {R2_test1:.2f}")

Test MSE for basic model = 0.22
Test R2 for basic model = 0.30


In [20]:
tmp = smf.ols(flexible_model, data=df)
X = tmp.data.exog
y = tmp.data.endog
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)

reg_flex = sm.OLS(y_train, X_train).fit()
yhat_reg_flex = reg_flex.predict(X_test)

MSE_test2 = np.mean((y_test - yhat_reg_flex) ** 2)
R2_test2 = 1 - MSE_test2 / np.var(y_test)

print(f"Test MSE for basic model = {MSE_test2:.4f}")
print(f"Test R2 for basic model = {R2_test2:.4f}")

Test MSE for basic model = 0.2742
Test R2 for basic model = 0.1204


In [21]:
lasso = Pipeline([("scale", StandardScaler()), ("lasso", LassoCV())])
lasso.fit(X_train[:, 1:], y_train)

yhat_reg_lasso = lasso.predict(X_test[:, 1:])

MSE_test3 = np.mean((y_test - yhat_reg_lasso)**2)
R2_test3 = 1. - MSE_test3 / np.var(y_test)

print(f"Test MSE for the lasso model: {MSE_test3}")
print(f"Test R2 for the lasso model: {R2_test3}")

/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.14052360610401138, tolerance: 0.10846290648765872
  model = cd_fast.enet_coordinate_descent_gram(
/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.43890193283141343, tolerance: 0.10846290648765872
  model = cd_fast.enet_coordinate_descent_gram(
/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8653933599593984, tolerance: 0.10846290648765

Test MSE for the lasso model: 0.22594930005846384
Test R2 for the lasso model: 0.275292858650252


In [22]:
res_df = pd.DataFrame(
    dict(
        model = ["basic reg", "flexbiel reg", "flexible lasso"],
        mse_test = [MSE_test1, MSE_test2, MSE_test3],
        r2_test = [R2_test1, R2_test2, R2_test3]
    )
)

res_df

,model,mse_test,r2_test
0,basic reg,0.215955,0.295597
1,flexbiel reg,0.274243,0.120397
2,flexible lasso,0.225949,0.275293


### Wage Gab based on Sex

https://colab.research.google.com/github/CausalAIBook/MetricsMLNotebooks/blob/main/PM1/python-ols-and-lasso-for-wage-gap-inference.ipynb

In [23]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
df = pd.read_csv(file)

In [31]:
# male: sex == 0
# female: sex == 1

df.groupby("sex").agg("mean")

,wage,lwage,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
sex,,,,,,,,,,,,,,,,,,,
0,24.019261,2.987830,0.031807,0.294303,0.273331,0.293953,0.106606,0.259000,0.298148,0.220902,0.221950,13.783992,2.991466,8.100175,24.598909,6121.125481,12.700454,6061.956309,12.037050
1,22.649413,2.949485,0.012669,0.180865,0.283967,0.347313,0.175186,0.260376,0.294452,0.210135,0.235037,13.731324,3.053246,8.405467,25.766892,4297.840979,10.384010,7338.091306,14.916557


In [38]:
#female - male
df.query("sex==1")["lwage"].mean() - df.query("sex==0")["lwage"].mean()

-0.03834473367441449

In [40]:
smf.ols("lwage ~ sex", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.752
Date:                Wed, 06 Mar 2024   Prob (F-statistic):             0.0165
Time:                        14:49:12   Log-Likelihood:                -4412.7
No. Observations:                5150   AIC:                             8829.
Df Residuals:                    5148   BIC:                             8843.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9878      0.011    280.316      0.000       2.967       3.009
sex           -0.0383      0.016     -2.398      0.017      -0.070      -0.007
==============================================================================
Omnibus:                      236.523   Durbin-Watson:                   1.733
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              406.245
Skew:                           0.376   Prob(JB):                     6.09e-89
Kurtosis:                       4.153   Cond. No.                         2.51
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [55]:
(alt.Chart(df.sample(5_000),
          title="Log wage for male (sex=0) and female (sex=1)")
    .mark_boxplot()
    .encode(x="sex:N", y="lwage")
    .properties(width=200, height=200)
)

alt.Chart(...)

In [81]:
(alt.Chart(df.sample(5_000),
          title="Log wage for male (sex=0) and female (sex=1)")
    .transform_density("lwage", as_=["lwage", "density"], extent=[0, 5], groupby=["sex"])
    .mark_area(orient="horizontal")
    .encode(alt.X("density:Q").stack("center").impute(None).title(None).axis(labels=False, values=[0], grid=False, ticks=True),
            alt.Y("lwage:Q"),
            alt.Column("sex").header(titleOrient="bottom", labelOrient="bottom", labelPadding=0),
            alt.Color("sex:N"))
    .configure_view(stroke=None)
)

alt.Chart(...)

Flexible model

In [82]:
flex = "lwage ~ sex + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)"

control_fit = smf.ols(flex, data=df).fit()
control_est = control_fit.params["sex"]
control_se = control_fit.HC3_se["sex"]

control_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     10.83
Date:                Wed, 06 Mar 2024   Prob (F-statistic):          2.69e-305
Time:                        15:05:00   Log-Likelihood:                -3301.9
No. Observations:                5150   AIC:                             7096.
Df Residuals:                    4904   BIC:                             8706.
Df Model:                         245                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              3.8603      0.429      9.006      0.000       3.020       4.701
C(occ2)[T.2]           0.1613      0.130      1.244      0.214      -0.093       0.416
C(occ2)[T.3]           0.2102      0.169      1.246      0.213      -0.121       0.541
C(occ2)[T.4]           0.0709      0.184      0.386      0.700      -0.289       0.431
C(occ2)[T.5]          -0.3960      0.189     -2.100      0.036      -0.766      -0.026
C(occ2)[T.6]          -0.2311      0.187     -1.236      0.217      -0.598       0.135
C(occ2)[T.7]           0.3147      0.194      1.621      0.105      -0.066       0.695
C(occ2)[T.8]          -0.1875      0.169     -1.108      0.268      -0.519       0.144
C(occ2)[T.9]          -0.3390      0.167     -2.027      0.043      -0.667      -0.011
C(occ2)[T.10]          0.0210      0.156      0.134      0.893      -0.286       0.328
C(occ2)[T.11]         -0.6424      0.309     -2.078      0.038      -1.248      -0.036
C(occ2)[T.12]         -0.0675      0.252     -0.268      0.789      -0.562       0.427
C(occ2)[T.13]         -0.2330      0.232     -1.006      0.314      -0.687       0.221
C(occ2)[T.14]          0.2562      0.323      0.794      0.427      -0.376       0.889
C(occ2)[T.15]         -0.1939      0.260     -0.747      0.455      -0.703       0.315
C(occ2)[T.16]         -0.0551      0.147     -0.375      0.708      -0.343       0.233
C(occ2)[T.17]         -0.4156      0.136     -3.053      0.002      -0.682      -0.149
C(occ2)[T.18]         -0.4822      1.044     -0.462      0.644      -2.530       1.565
C(occ2)[T.19]         -0.2579      0.333     -0.776      0.438      -0.910       0.394
C(occ2)[T.20]         -0.3010      0.234     -1.286      0.199      -0.760       0.158
C(occ2)[T.21]         -0.4272      0.221     -1.936      0.053      -0.860       0.005
C(occ2)[T.22]         -0.8695      0.298     -2.922      0.003      -1.453      -0.286
C(ind2)[T.3]          -1.2474      0.645     -1.932      0.053      -2.513       0.018
C(ind2)[T.4]          -0.0948      0.464     -0.205      0.838      -1.004       0.814
C(ind2)[T.5]          -0.5294      0.435     -1.218      0.223      -1.381       0.323
C(ind2)[T.6]          -0.6222      0.435     -1.431      0.152      -1.474       0.230
C(ind2)[T.7]          -0.5047      0.502     -1.005      0.315      -1.490       0.480
C(ind2)[T.8]          -0.7295      0.467     -1.561      0.119      -1.646       0.187
C(ind2)[T.9]          -0.8025      0.425     -1.887      0.059      -1.636       0.031
C(ind2)[T.10]         -0.5806      0.481     -1.207      0.227      -1.523       0.362
C(ind2)[T.11]         -0.9852      0.448     -2.198      0.028      -1.864      -0.107
C(ind2)[T.12]         -0.7376      0.424     -1.738      0.082      -1.569       0.094
C(ind2)[T.13]         -1.0183      0.4

In [86]:
print(f"The estimated sex coefficient is {control_est:.4f} with robust SE of {control_se:.4f}")

The estimated sex coefficient is -0.0696 with robust SE of 0.0157


In [100]:
flex_without_sex = "lwage ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)"
lm0 = smf.ols(flex_without_sex, data=df.query("sex==0"))
lm1 = smf.ols(flex_without_sex, data=df.query("sex==1"))

XX0 = lm0.exog
y0 = lm0.endog
XX1 =  lm1.exog
y1 = lm1.endog

betarest = control_fit.params[control_fit.params.index!="sex"][1:]

print(f"Marginal gap = {y1.mean() - y0.mean()}")
#diff da media das fts (exclui intercept '[1:]')
diff_explained = betarest.dot(XX1.mean(0)[1:] - XX0.mean(0)[1:])
print(f"Explained difference: {diff_explained}")
print(f"Unexplained difference: {control_est}")
print(f"Sum of differences: {diff_explained + control_est}")

Marginal gap = -0.03834473367441449
Explained difference: 0.03120846962241225
Unexplained difference: -0.06955320329684149
Sum of differences: -0.03834473367442924


Frisch-Waugh-Lovell

In [101]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

flex_interact_sex = "lwage ~ sex*((exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we))"
lmi = smf.ols(flex_interact_sex, data=df)

vfi = np.array([variance_inflation_factor(lmi.exog, i) for i in range(1, lmi.exog.shape[1])])


/Users/danieloliveiradebrito/Projetos/causal-inference/venv/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [121]:
# Concatena False para remover intercept
np.array(lmi.exog_names)[np.concatenate(((False, ), vfi>1e8))]

array(['C(occ2)[T.18]', 'sex:C(occ2)[T.18]', 'exp1:C(occ2)[T.18]',
       'exp2:C(occ2)[T.18]', 'exp3:C(occ2)[T.18]', 'exp4:C(occ2)[T.18]',
       'sex:exp1:C(occ2)[T.18]', 'sex:exp2:C(occ2)[T.18]', 'sex:exp3',
       'sex:exp3:C(occ2)[T.18]', 'sex:exp4:C(occ2)[T.18]'], dtype='<U22')

In [123]:
flex_y = "lwage ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)"
flex_d = "sex ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)"

t_Y = smf.ols(flex_y, data=df).fit().resid
t_D = smf.ols(flex_d, data=df).fit().resid

partial_fit = sm.OLS(t_Y, t_D).fit()
partial_est = partial_fit.params["x1"]

print(f"Coefficient for D via partialling-out {partial_est:.4f}")

partial_se = partial_fit.HC3_se["x1"]
print(f"95% CI = {partial_fit.conf_int().values[0]}")

Coefficient for D via partialling-out -0.0696
95% CI = [-0.09866859 -0.04043781]


In [147]:
chart = alt.Chart(pd.DataFrame(
    dict(
        y_resid = t_Y,
        d_resid = t_D
    )
).sample(5_000),
title="Linear regression Y_resid ~ D_resid").mark_point(opacity=0.1).encode(y="y_resid", x="d_resid")

chart + chart.transform_regression("d_resid", "y_resid").mark_line()

alt.LayerChart(...)

## Study Questions

### 1. Sample splitting to evaluate the performance of predictions

The concept of sample splitting for evaluating prediction performance involves dividing the data into two sets: one for training the model and another for testing it. This allows us to gauge how well our trained model performs on unseen data, or out-of-sample data, providing an estimation of its generalization capability. This step is crucial in machine learning, especially for models with high flexibility, as they may inadvertently learn from noise patterns in the training data that do not generalize well to new data.

Consider a scenario where we have a linear relationship $Y_i = \beta X_i + \epsilon_i$, with $\epsilon \sim N(0, 1)$. In this case, training a highly flexible model could lead it to capture the noise from $\epsilon$ rather than the true underlying pattern. This model would have a perfect fit on the training data ($MSE = 0$), but a terrible performance on out-of-sample data.

In [196]:
true_beta = 2
x = np.arange(0, 5, step = .05)
n_samples = len(x)
random_noise = np.random.normal(0, 1, n_samples)
y = 5 + true_beta * x + random_noise


In [197]:
from sklearn.tree import DecisionTreeRegressor

dt_reg = DecisionTreeRegressor()
dt_reg.fit(x.reshape(-1, 1), y)

lm = sm.OLS(y, sm.add_constant(x.reshape(-1, 1))).fit()


df_pred = pd.DataFrame(dict(x=x, y=y, y_lm = lm.predict(sm.add_constant(x.reshape(-1, 1))), y_dt = dt_reg.predict(x.reshape(-1, 1))))

base = alt.Chart(df_pred, title=f"Compara OLS (valor estimado de {lm.params[1]:.4f}) com DT")


base.mark_point().encode(alt.X("x"), alt.Y("y")) + base.mark_line().encode(alt.X("x"), alt.Y("y_dt")) + base.mark_line(color="red", strokeDash=[2, 2]).encode(alt.X("x"), alt.Y("y_lm"))

alt.LayerChart(...)

### 2. Wage gap analysis, focusing on subset of college-educated workers

In [3]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
df = pd.read_csv(file)
df_college = df.query("clg == 1")

In [11]:
flex_y = "lwage ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+C(occ2)+C(ind2)+mw+so+we)"
flex_d = "sex ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+C(occ2)+C(ind2)+mw+so+we)"

t_Y = smf.ols(flex_y, data=df_college).fit().resid
t_D = smf.ols(flex_d, data=df_college).fit().resid

partial_fit = sm.OLS(t_Y, t_D).fit()
partial_est = partial_fit.params["x1"]

print(f"Coefficient for D via partialling-out {partial_est:.4f}")

partial_se = partial_fit.HC3_se["x1"]
print(f"95% CI = {partial_fit.conf_int().values[0]}")

Coefficient for D via partialling-out -0.0462
95% CI = [-0.09291928  0.00047468]


In [13]:
chart = alt.Chart(pd.DataFrame(
    dict(
        y_resid = t_Y,
        d_resid = t_D
    )
), title="Linear regression Y_resid ~ D_resid (college educated)").mark_point(opacity=.2).encode(x=alt.X("d_resid"), y=alt.Y("y_resid"))

#chart + chart.transform_regression("d_resid", "y_resid").mark_line()
chart + chart.transform_loess("d_resid", "y_resid").mark_line()

alt.LayerChart(...)

### 3. Linear regression with $n = 2$, $p = 1$ ($Y_i = \hat{\beta}X_i + \hat{\epsilon_i}$). Geometric interpretation of $\hat Y$

https://www.youtube.com/watch?v=PbyP3goun2Y